# Active Learning

## Preporcessing

In [110]:
# !pip install modAL-python
# !pip install category_encoders
# !pip install xgboost

In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
import numpy as np
import warnings
warnings.filterwarnings("ignore")




# Read dataset
df = pd.read_csv('Dataset-vf.csv', sep=',', header=0)

# Display dataset
display(df)

,X5,X2,X3,X4,X6,X1,X10,X7,X8,X9,X11,X12,X15,X13,X14,Y
0,43,59.0,11,120,150,2709,2114,228,217,120,0.66,269,R1,FC11,CA49,BMW
1,0,54.0,7,0,693,2802,162,224,225,136,0.35,195,NaN,FC11,CA50,BMW
2,0,28.0,12,0,1260,2325,162,215,213,133,0.89,750,NaN,FC14,CA26,Mercedes
3,16,112.0,8,272,3649,2837,6221,235,231,128,0.27,-155,NaN,FC11,CA32,Ford
4,10,59.0,7,134,900,2509,5184,226,226,134,0.93,88,NaN,FC11,CA38,BMW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6119,14,NaN,18,67,918,2712,1922,201,254,183,0.00,-877,NaN,FC13,CA24,Toyota
6120,1,67.0,6,170,824,3101,1657,227,228,135,0.50,437,NaN,FC11,CA43,Toyota
6121,-3,139.0,13,30,3127,3050,5770,240,235,123,0.74,53,NaN,FC11,CA40,Toyota
6122,12,106.0,24,30,582,2080,108,252,202,64,0.88,-79,NaN,FC14,CA23,Mercedes


In [112]:
# Get nan statistics
print(df.isnull().sum())


X5        0
X2      604
X3        0
X4        0
X6        0
X1        0
X10       0
X7        0
X8        0
X9        0
X11       0
X12       0
X15    5523
X13       0
X14       0
Y         0
dtype: int64


In [113]:
# drop column X15
df = df.drop(['X15'], axis=1)

# interpolate nan values in X2
df['X2'] = df['X2'].interpolate(method='linear')

# Display dataset
display(df)

# Get nan statistics
print(df.isnull().sum())

,X5,X2,X3,X4,X6,X1,X10,X7,X8,X9,X11,X12,X13,X14,Y
0,43,59.0,11,120,150,2709,2114,228,217,120,0.66,269,FC11,CA49,BMW
1,0,54.0,7,0,693,2802,162,224,225,136,0.35,195,FC11,CA50,BMW
2,0,28.0,12,0,1260,2325,162,215,213,133,0.89,750,FC14,CA26,Mercedes
3,16,112.0,8,272,3649,2837,6221,235,231,128,0.27,-155,FC11,CA32,Ford
4,10,59.0,7,134,900,2509,5184,226,226,134,0.93,88,FC11,CA38,BMW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6119,14,60.0,18,67,918,2712,1922,201,254,183,0.00,-877,FC13,CA24,Toyota
6120,1,67.0,6,170,824,3101,1657,227,228,135,0.50,437,FC11,CA43,Toyota
6121,-3,139.0,13,30,3127,3050,5770,240,235,123,0.74,53,FC11,CA40,Toyota
6122,12,106.0,24,30,582,2080,108,252,202,64,0.88,-79,FC14,CA23,Mercedes


X5     0
X2     0
X3     0
X4     0
X6     0
X1     0
X10    0
X7     0
X8     0
X9     0
X11    0
X12    0
X13    0
X14    0
Y      0
dtype: int64


In [114]:
# get unique values of all columns
for col in df.columns[-3:]:
    print(col, df[col].unique())

X13 [' FC11' ' FC14' ' FC13' ' FC12']
X14 [' CA49' ' CA50' ' CA26' ' CA32' ' CA38' ' CA42' ' CA37' ' CA23' ' CA24'
 ' CA21' ' CA43' ' CA22' ' CA36' ' CA33' ' CA44' ' CA59' ' CA30' ' CA51'
 ' CA52' ' CA31' ' CA40' ' CA58' ' CA25' ' CA39' ' CA53' ' CA60' ' CA46'
 ' CA34' ' CA41' ' CA48' ' CA54' ' CA55']
Y ['BMW' 'Mercedes' 'Ford' 'Toyota' 'Honda' 'GMC']


In [115]:
# one hot encoding for X13 and Y
df = pd.get_dummies(df, columns=['X13', 'Y'])

y_categorical = np.argmax(df.iloc[:, -6:].values, axis=1)
print(y_categorical)

# binary encoding for X14
encoder = ce.BinaryEncoder(cols=['X14'])
df = encoder.fit_transform(df)

# Display dataset
display(df.columns)

[0 0 4 ... 5 4 5]


Index(['X5', 'X2', 'X3', 'X4', 'X6', 'X1', 'X10', 'X7', 'X8', 'X9', 'X11',
       'X12', 'X14_0', 'X14_1', 'X14_2', 'X14_3', 'X14_4', 'X14_5',
       'X13_ FC11', 'X13_ FC12', 'X13_ FC13', 'X13_ FC14', 'Y_BMW', 'Y_Ford',
       'Y_GMC', 'Y_Honda', 'Y_Mercedes', 'Y_Toyota'],
      dtype='object')

## Least Confidence Sampling

In [127]:
# get least confident instances from unlabeled data
def lease_confidence(model, X_unlabeled, n_instances=1):
    # get probability of each instance
    proba = model.predict_proba(X_unlabeled)

    # get the least confident instances
    least_confidence = np.argsort(proba.max(axis=1))[:n_instances]

    indices = X_unlabeled.index[least_confidence].tolist()
    indices_values = X_unlabeled.index[least_confidence].tolist()[0]

    return indices, indices_values

## Entropy Sampling

In [144]:
def entropy_sampling(model, X_unlabeled, n_instances=1):
    # get probability of each instance
    proba = model.predict_proba(X_unlabeled)

    # get the least confident instances
    entropy_val = np.argsort(entropy(proba, axis=1))[:n_instances]

    indices = X_unlabeled.index[entropy_val].tolist()
    indices_values = X_unlabeled.index[entropy_val].tolist()[0]

    return indices, indices_values

In [151]:
# initialize active learneing loop 
def active_learning(model, x_labeled, y_labeled, x_unlabeled, y_unlabeled, query_strategy = 'entropy', budget = 500):
    added_labels = 0
    macro_f1 = 0
    while added_labels < budget:
        if query_strategy == 'entropy':
            query_confidence, query_idx = entropy_sampling(model, x_unlabeled)
        elif query_strategy == 'least_confidence':
            query_confidence, query_idx = lease_confidence(model, x_unlabeled)
        else:
            print("Invalid query strategy")
            break
        # add queried instances to labeled data
        x_labeled = pd.concat([x_labeled, x_unlabeled.iloc[query_idx]])
        y_labeled = pd.concat([y_labeled, y_unlabeled.iloc[query_idx]])
        display(x_labeled)
        display(y_labeled)
        # remove queried instances from unlabeled data
        x_unlabeled = x_unlabeled.drop(x_unlabeled.index[query_idx])
        y_unlabeled = y_unlabeled.drop(y_unlabeled.index[query_idx])
        
        # train model on new labeled data
        model.fit(x_labeled, y_labeled)
        y_pred = model.predict(x_unlabeled)
        macro_f1 = f1_score(y_unlabeled, y_pred, average='macro')
        added_labels += 1
        print("Added labels: ", added_labels)
        print("Macro f1 score: ", macro_f1)
    return model, macro_f1

In [152]:
import xgboost as xgb
X = df.iloc[:, :-6]
y = pd.DataFrame(y_categorical)

display(X)
display(y)

# split data into labeled and unlabeled
x_labeled, x_unlabeled, y_labeled, y_unlabeled = train_test_split(X, y, train_size=100, random_state=42)

# hyperparameters for the model
parameters = {
    'max_depth': 6,
    'eta': 0.3,
    'objective': 'multi:softprob',
    'num_class': 2,
    'random_state': 42,
    'enable_categorical': True,
    }


# initialize the model
model = xgb.XGBClassifier(**parameters)
model.fit(x_labeled, y_labeled)
y_pred = model.predict(x_unlabeled)
y_pred = y_pred.astype(bool)
print(y_pred.shape, y_unlabeled.shape)
macro_f1 = f1_score(y_unlabeled, y_pred, average='macro')
print("Initial macro f1 score: ", macro_f1)

,X5,X2,X3,X4,X6,X1,X10,X7,X8,X9,...,X14_0,X14_1,X14_2,X14_3,X14_4,X14_5,X13_ FC11,X13_ FC12,X13_ FC13,X13_ FC14
0,43,59.0,11,120,150,2709,2114,228,217,120,...,0,0,0,0,0,1,True,False,False,False
1,0,54.0,7,0,693,2802,162,224,225,136,...,0,0,0,0,1,0,True,False,False,False
2,0,28.0,12,0,1260,2325,162,215,213,133,...,0,0,0,0,1,1,False,False,False,True
3,16,112.0,8,272,3649,2837,6221,235,231,128,...,0,0,0,1,0,0,True,False,False,False
4,10,59.0,7,134,900,2509,5184,226,226,134,...,0,0,0,1,0,1,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6119,14,60.0,18,67,918,2712,1922,201,254,183,...,0,0,1,0,0,1,False,False,True,False
6120,1,67.0,6,170,824,3101,1657,227,228,135,...,0,0,1,0,1,1,True,False,False,False
6121,-3,139.0,13,30,3127,3050,5770,240,235,123,...,0,1,0,1,0,1,True,False,False,False
6122,12,106.0,24,30,582,2080,108,252,202,64,...,0,0,1,0,0,0,False,False,False,True


,0
0,0
1,0
2,4
3,1
4,0
...,...
6119,5
6120,5
6121,5
6122,4


(6024,) (6024, 1)
Initial macro f1 score:  0.14292274317055845


In [153]:
# initialize active learning loop
entropy_model, entropy_macro_f1 = active_learning(model, x_labeled, y_labeled, x_unlabeled, y_unlabeled, query_strategy='entropy', budget=500)

least_confidence_model, least_confidence_macro_f1 = active_learning(model, x_labeled, y_labeled, x_unlabeled, y_unlabeled, query_strategy='least_confidence', budget=500)

print("Entropy macro f1 score: ", entropy_macro_f1)
print("Least confidence macro f1 score: ", least_confidence_macro_f1)

# plot macro f1 score
plt.plot(entropy_macro_f1, label='entropy')
plt.plot(least_confidence_macro_f1, label='least confidence')
plt.xlabel('Number of added labels')
plt.ylabel('Macro f1 score')
plt.legend()
plt.show()

,X5,X2,X3,X4,X6,X1,X10,X7,X8,X9,...,X14_1,X14_2,X14_3,X14_4,X14_5,X13_ FC11,X13_ FC12,X13_ FC13,X13_ FC14,0
5463,198.0,135.0,35.0,283.0,960.0,2311.0,408.0,249.0,199.0,46.0,...,0.0,1.0,0.0,0.0,0.0,False,False,False,True,NaN
2568,14.0,121.0,21.0,42.0,976.0,2207.0,1315.0,250.0,218.0,85.0,...,0.0,1.0,0.0,0.0,0.0,False,False,False,True,NaN
64,1.0,101.0,5.0,30.0,900.0,2047.0,541.0,228.0,233.0,139.0,...,0.0,0.0,1.0,1.0,1.0,False,False,False,True,NaN
3884,22.0,114.0,6.0,180.0,192.0,2284.0,285.0,230.0,234.0,138.0,...,1.0,0.0,0.0,0.0,1.0,False,False,False,True,NaN
2062,51.0,224.0,22.0,108.0,2266.0,2812.0,680.0,184.0,254.0,199.0,...,0.0,1.0,1.0,1.0,0.0,False,False,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X14_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
X13_ FC11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
X13_ FC12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
X13_ FC13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


,0
5463,4
2568,4
64,4
3884,4
2062,0
...,...
5191,4
5226,4
5390,4
860,3


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:X13_ FC11: object, X13_ FC12: object, X13_ FC13: object, X13_ FC14: object, 0: object